# Stereo Panning Spectrum

In [29]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy import signal
import pandas as pd
plt.style.use('dark_background')

#Variables

filename = 'Runawaytest.wav' 
fs = 22050
N = 512
H = 256
L=2*N

#Define Function
def extract_SPSF(filename, fs, N, H, L):
    y, sr = librosa.load(filename, mono=False)
    y_left=np.asfortranarray(y[0])
    y_right=np.asfortranarray(y[1])
    
    stft_left = np.abs(librosa.stft(y_left, n_fft=L, hop_length=H, win_length=N,
                                window='hann', pad_mode='constant', center=True)) 
    stft_right = np.abs(librosa.stft(y_right,n_fft=L, hop_length=H, win_length=N,
                                window='hann', pad_mode='constant', center=True))
    
    
    ##### Define functions and equations
    def similarity(stft_left, stft_right): ## Equation [1]
        num = np.multiply(stft_left,np.conj(stft_right)) 
        den = np.add(np.square(stft_left), np.square(stft_right))    
        sim = np.divide(num, den) 
        return np.multiply(2,sim)

    def similarity_left(stft_left, stft_right):  #Equation [3]
        num = np.multiply(stft_left,np.conj(stft_right))
        den = np.square(stft_left)
        sim_left = np.divide(num, den) 
        return sim_left

    def similarity_right(stft_left, stft_right): #Equation [3]
        num = np.multiply(stft_left, np.conj(stft_right))
        den = np.square(stft_right)
        sim_right = np.divide(num, den) 
        return sim_right

    Vl = similarity_left(stft_left, stft_right) #Equation [3]
    Vr = similarity_right(stft_left, stft_right) #Equation [3]
    V = similarity(stft_left, stft_right) ## Equation [1]

    delta = np.subtract(Vl, Vr)  #Equation [4]
    delta_sign = np.sign(delta) #Equation[5]
    sps = np.multiply(np.subtract(1, V), delta_sign) #Equation[6]

    rms = librosa.feature.rmse(S=sps) #Equation[7]

    #get frequency bands
    sps_high=sps[58:512] #250 to 22.050
    sps_mid=sps[6:58] #250 to 2500
    sps_low=sps[0:6] #0 to 250Hz

    rms_low = librosa.feature.rmse(S=sps_low)
    rms_mid = librosa.feature.rmse(S=sps_mid)
    rms_high = librosa.feature.rmse(S=sps_high)

    #join all to create 4d vector
    phi = np.array([rms, rms_low, rms_mid, rms_high]) #Equation[8]
    phi = np.reshape(phi, (phi.shape[0],phi.shape[2]) )

    #pandas to calculate mean and std
    phi = pd.DataFrame(data=phi)
    
    #mean and std
    P = phi.T.rolling(40, min_periods=1).agg({np.std, np.mean})
    
    #Final 16D vector
    Grand_mean = P.mean()#df['grand']
    Grand_std = P.std()#df['grand']
    SPSF = np.concatenate((Grand_mean.to_numpy(), Grand_std.to_numpy())) #Final 16d Vector
    return SPSF

def extract_mfcc(filename):
    audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
    mfccsscaled = np.mean(mfccs.T,axis=0)
    return mfccsscaled[2:] #remove first two



In [30]:

filename = 'Runawaytest.wav' 
fs = 22050
N = 512
H = 256
L=2*N

SPSF= extract_SPSF(filename, fs, N, H, L)
mfccs = extract_mfcc(filename)

print(SPSF)
print(len(SPSF))

print(mfccs)
print(len(mfccs))



[0.07381101 0.31246693 0.09077734 0.11481843 0.06920551 0.26039386
 0.07833913 0.31719505 0.06342063 0.12182509 0.05216973 0.10515662
 0.02603356 0.09108676 0.06727919 0.1274653 ]
16
[-6.31095598 24.98901718 -3.10023135 22.49716211  1.27341129 15.96748261
 -2.80335784  7.63726402  5.11017266 12.9886806   1.18004774]
11
